### Joining Longitdue/Latitude data for Geometric Heat Map

In [12]:
import pandas as pd
import sqlite3


df = pd.read_csv('housing_data_tableau_ready_FIXED.csv')
df_1 = pd.read_excel('coordinates.xlsx')

conn = sqlite3.connect("coordinates.db")
cursor = conn.cursor()

df.to_sql("indexes", conn, if_exists="replace", index=False)
df_1.to_sql("coordinates", conn, if_exists="replace", index=False)

df_2 = """ DROP TABLE IF  EXISTS I_C;
CREATE TABLE I_C AS
SELECT i.*, c.LONG, c.LAT FROM indexes i LEFT JOIN coordinates c ON i.District = c.LAD24NM; 
"""
print(pd.read_sql("SELECT DISTINCT i.District, c.LONG, c.LAT FROM indexes i LEFT JOIN coordinates c ON i.District = c.LAD24NM  LIMIT 20; ", conn))

cursor.executescript(df_2)
conn.commit()

# Checking for inconsistencies between  
print(pd.read_sql("SELECT COUNT(DISTINCT District) FROM I_C WHERE LAT IS NOT NULL AND LONG IS NOT NULL GROUP BY District", conn))

print(pd.read_sql("SELECT DISTINCT LAT FROM I_C LIMIT 20", conn))

# Checking for districts in indexes not in coordinates
print(pd.read_sql("""
                  SELECT DISTINCT i.District as index_districts_not_in_coordinates
FROM indexes i
LEFT JOIN coordinates c ON i.District = c.LAD24NM
WHERE c.LAD24NM IS NULL
ORDER BY i.District; """, conn))
# 11 districts with differing names - i will alter these 11 name in indexes manually

df_3 = pd.read_sql_query(""" SELECT * FROM I_C; """, conn)

df_3.to_csv('housing_data_tableau_ready_FIXED_GEO.csv', index=False)

conn.close()


                        District      LONG      LAT
0                           Adur -0.324170  50.8457
1                   Amber Valley -1.462190  53.0288
2                           Arun -0.649990  50.8432
3                       Ashfield -1.254220  53.0975
4                        Ashford  0.823374  51.1310
5                        Babergh  0.916149  52.0645
6           Barking and Dagenham  0.129479  51.5456
7                         Barnet -0.218190  51.6111
8                       Barnsley -1.549250  53.5258
9                       Basildon  0.475056  51.5904
10         Basingstoke and Deane -1.220230  51.2594
11                     Bassetlaw -0.978700  53.3560
12  Bath and North East Somerset -2.486570  51.3560
13                       Bedford -0.454630  52.1963
14                        Bexley  0.146212  51.4582
15                    Birmingham -1.881410  52.4840
16                         Blaby -1.198870  52.5771
17         Blackburn with Darwen -2.463610  53.7008
18          